In [ ]:
import os
from PIL import Image
import pandas as pd

# Load your dataset
csv_path = '/content/Sample_prize (1).csv'
df = pd.read_csv(csv_path)

# Make sure the directory exists
os.makedirs('/content/data', exist_ok=True)

# Create dummy image for each file in 'image_path'
for rel_path in df['image_path']:
    full_path = os.path.join('/content', rel_path)
    if not os.path.exists(full_path):
        img = Image.new('RGB', (224, 224), color=(255, 255, 255))  # white dummy image
        img.save(full_path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ✅ STEP 1: Install Required Libraries
!pip install transformers torchvision pandas scikit-learn -q

# ✅ STEP 2: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# ✅ STEP 3: Load and Prepare Dataset
csv_path = '/content/Sample_prize (1).csv'  # Your uploaded CSV path
df = pd.read_csv(csv_path)

# ✅ STEP 4: Custom Dataset
class PriceDataset(Dataset):
    def __init__(self, dataframe, image_dir, tokenizer, transform):
        self.df = dataframe
        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, os.path.basename(row['image_path']))
        image = self.transform(Image.open(img_path).convert("RGB"))
        encoding = self.tokenizer(row['title'], return_tensors='pt', padding='max_length', truncation=True, max_length=32)
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        price = torch.tensor(row['price'], dtype=torch.float32)
        return image, input_ids, attention_mask, price

# ✅ STEP 5: Transforms and Tokenizer
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# ✅ STEP 6: Split Data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_data = PriceDataset(train_df, '/content/data', tokenizer, transform)
val_data = PriceDataset(val_df, '/content/data', tokenizer, transform)
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=4)

# ✅ STEP 7: Define the Multimodal Model
from torchvision.models import resnet18, ResNet18_Weights

class PricePredictionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.image_model = resnet18(weights=ResNet18_Weights.DEFAULT)
        self.image_model.fc = nn.Identity()  # 512 features

        self.text_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.fc = nn.Sequential(
            nn.Linear(512 + 768, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, image, input_ids, attention_mask):
        img_feat = self.image_model(image)
        text_feat = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        combined = torch.cat((img_feat, text_feat), dim=1)
        price = self.fc(combined)
        return price.squeeze()

# ✅ STEP 8: Train the Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PricePredictionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_losses, val_losses = [], []

for epoch in range(5):
    model.train()
    total_loss = 0
    for image, input_ids, attention_mask, price in train_loader:
        image = image.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        price = price.to(device)

        optimizer.zero_grad()
        output = model(image, input_ids, attention_mask)
        loss = criterion(output, price)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    train_losses.append(total_loss / len(train_loader))

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for image, input_ids, attention_mask, price in val_loader:
            image = image.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            price = price.to(device)

            output = model(image, input_ids, attention_mask)
            loss = criterion(output, price)
            val_loss += loss.item()
    val_losses.append(val_loss / len(val_loader))

    print(f"Epoch {epoch+1}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

# ✅ STEP 9: Plot Training Curve
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Price Prediction Training")
plt.legend()
plt.show()

# ✅ STEP 10: Save the Model
torch.save(model.state_dict(), "/content/price_predictor.pth")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1: Train Loss = 177263.8969, Val Loss = 93600.4601


In [ ]:
# ✅ STEP 1: Install Required Libraries
!pip install transformers gradio torchvision pandas scikit-learn -q

# ✅ STEP 2: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel, BlipProcessor, BlipForConditionalGeneration
import os
import random
import gradio as gr
from sklearn.model_selection import train_test_split
from torchvision.models import resnet18, ResNet18_Weights

# ✅ STEP 3: Load Price Prediction Model
class PricePredictionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.image_model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 512)
        )

        self.text_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.fc = nn.Sequential(
            nn.Linear(512 + 768, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, image, input_ids, attention_mask):
        img_feat = self.image_model(image)
        text_feat = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        combined = torch.cat((img_feat, text_feat), dim=1)
        return self.fc(combined).squeeze()

# ✅ Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PricePredictionModel()
model.load_state_dict(torch.load("content/price_predictor.pth", map_location=device))
model.to(device)
model.eval()

# ✅ Tokenizer and Image Transform
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ Load BLIP for Title and Description
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
conditions = ["Like New", "Used", "Excellent", "Fair"]
product_list = []
cart = []

# ✅ Simple Session State for Access Control
user_session = {"logged_in": False, "role": None, "username": ""}

# ✅ Seller Upload

def generate_listing(image):
    if user_session["role"] != "Seller":
        return "❌ Only sellers can upload products."

    inputs = blip_processor(images=image, return_tensors="pt")
    out = blip_model.generate(**inputs)
    title = blip_processor.decode(out[0], skip_special_tokens=True).capitalize()
    description = f"This is a {random.choice(conditions)} item titled '{title}'. High quality and fast shipping guaranteed."

    transformed = transform(image).unsqueeze(0).to(device)
    encoded = tokenizer(title, return_tensors="pt", padding="max_length", truncation=True, max_length=32)
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)

    with torch.no_grad():
        price = model(transformed, input_ids, attention_mask).item()

    product = {
        "title": title,
        "description": description,
        "price": round(price, 2),
        "image": image
    }
    product_list.append(product)

    return f"\n✅ Title: {title}\n📄 Description: {description}\n💲 Price: ${round(price, 2)}"

# ✅ AI Agent Buyer Chat

def ai_agent_recommendation(message):
    if user_session["role"] != "Buyer":
        return "❌ Only buyers can use the AI agent.", None

    if not product_list:
        return "⚠️ No products available right now.", None

    query = message.lower()
    recommendations = [p for p in product_list if query in p["title"].lower() or query in p["description"].lower()]

    if not recommendations:
        selected = random.choice(product_list)
        text = f"🤖 I couldn't find an exact match. Here's a random pick:\n📌 {selected['title']} - ${selected['price']}\n{selected['description']}"
    else:
        selected = recommendations[0]
        text = f"✅ Recommended: \n📌 {selected['title']} - ${selected['price']}\n{selected['description']}"

    cart.append(selected)
    return text + f"\n🛒 Added to your cart.", selected['image']

def view_cart():
    if user_session["role"] != "Buyer":
        return "❌ Only buyers can view cart.", None

    if not cart:
        return "🛒 Your cart is empty.", None
    total = sum(p["price"] for p in cart)
    items = "\n".join([f"{p['title']} - ${p['price']}" for p in cart])
    images = [p["image"] for p in cart]
    return f"🛍️ Cart Items:\n{items}\n\n💰 Total: ${round(total, 2)}", images

# ✅ Shipping & Notifications

def generate_shipping_label(name, address):
    tracking_id = f"TRK{random.randint(100000, 999999)}"
    return f"""
🚚 SHIPPING LABEL
------------------------
To: {name}
Address: {address}
Tracking ID: {tracking_id}
Carrier: AI Express
ETA: 3–5 business days
""".strip()

def track_order(tracking_id):
    statuses = [
        "📦 Order Placed",
        "🚚 Dispatched",
        "🚛 In Transit",
        "🏙️ At Destination Hub",
        "📬 Out for Delivery",
        "✅ Delivered"
    ]
    return "\n".join(statuses[:random.randint(1, len(statuses))])

def notify_user(email, message):
    return f"📩 Simulated email sent to {email}:\n'{message}'"

# ✅ Login Functionality
users_db = {
    "seller1": "pass123",
    "buyer1": "pass456"
}

def login_user(username, password, role):
    if username in users_db and users_db[username] == password:
        user_session["logged_in"] = True
        user_session["role"] = role
        user_session["username"] = username
        return f"✅ Logged in as {username} ({role})"
    return "❌ Invalid credentials"

# ✅ Gradio App
with gr.Blocks() as demo:
    gr.Markdown("## 🛒 AI Marketplace – Login to Access Seller/Buyer Panels")

    with gr.Tab("🔐 Login"):
        uname = gr.Textbox(label="Username")
        pword = gr.Textbox(label="Password", type="password")
        role = gr.Dropdown(["Seller", "Buyer"], label="Login As")
        login_result = gr.Textbox()
        login_btn = gr.Button("Login")
        login_btn.click(fn=login_user, inputs=[uname, pword, role], outputs=login_result)

    with gr.Tab("📤 Seller Upload"):
        gr.Markdown("Only accessible if logged in as Seller")
        image_input = gr.Image(type="pil")
        listing_output = gr.Textbox(label="AI Listing")
        upload_btn = gr.Button("Upload Product")
        upload_btn.click(fn=generate_listing, inputs=image_input, outputs=listing_output)

    with gr.Tab("🤖 Buyer AI Agent"):
        gr.Markdown("Interact with our AI shopping agent to find and purchase items!")
        ai_chat = gr.Textbox(label="What do you want to buy?")
        ai_reply = gr.Textbox(label="AI Agent Response")
        ai_img = gr.Image(label="Product Preview")
        ai_button = gr.Button("Ask AI")
        ai_button.click(fn=ai_agent_recommendation, inputs=ai_chat, outputs=[ai_reply, ai_img])

        gr.Markdown("### 🧾 View Cart")
        cart_output = gr.Textbox()
        cart_images = gr.Gallery(label="Cart Images", show_label=False)
        cart_btn = gr.Button("View Cart")
        cart_btn.click(fn=view_cart, outputs=[cart_output, cart_images])

    with gr.Tab("📦 Shipping & Tracking"):
        gr.Markdown("### 📤 Generate Shipping Label")
        name = gr.Textbox(label="Recipient Name")
        address = gr.Textbox(label="Shipping Address")
        label_out = gr.Textbox(label="Shipping Label")
        gen_label_btn = gr.Button("Generate Label")
        gen_label_btn.click(fn=generate_shipping_label, inputs=[name, address], outputs=label_out)

        gr.Markdown("### 📍 Track Order")
        tracking_in = gr.Textbox(label="Tracking ID")
        status_out = gr.Textbox(label="Order Status")
        track_btn = gr.Button("Track")
        track_btn.click(fn=track_order, inputs=tracking_in, outputs=status_out)

        gr.Markdown("### 📢 Notifications")
        email = gr.Textbox(label="Email")
        note = gr.Textbox(label="Message")
        note_out = gr.Textbox(label="Notification Result")
        notify_btn = gr.Button("Send")
        notify_btn.click(fn=notify_user, inputs=[email, note], outputs=note_out)

# ✅ Launch
demo.launch(share=True)
